# Music segmentation

<!-- {{ add_binder_block(page) }} -->

## Introduction

Music segmentation can be seen as a change point detection task and therefore can be carried out with `ruptures`.
Roughly, it consists of finding the temporal boundaries of homogeneous segments, e.g. the intro, verse, chorus and outro in a song.

The adopted approach is summarized as follows:

- the original sound is transformed into an informative (multivariate) representaion;
- mean shifts are detected in this new representation using a dynamic programming approach.

In this example, we try two representations which are well-known in the sound and music processing community:

- a chromagram, that maps the input signal into a 12 features space representing the classical 12 pitch classes;
- a tempogram computed based on the onset strength envelope of the input signal, that captures the tempo spectrum.

To load and manipulate sound data, we use the [librosa package](https://librosa.org/doc/latest/index.html).

## Setup

First, we make the necessary imports.

In [ ]:
from itertools import chain, product

import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import ruptures as rpt  # our package
from IPython.display import Audio, display

We can also define a few utility functions.

In [ ]:
def fig_ax(figsize=(15, 5)):
    """Return a (matplotlib) figure and ax objects with given size."""
    return plt.subplots(figsize=figsize)

## Load the data

A number of music files are available in [Librosa](https://librosa.org/doc/latest/index.html).
See [here](https://librosa.org/doc/latest/recordings.html#description-of-examples) for a complete list.
In this example, we choose the *Dance of the Sugar Plum Fairy* from *The Nutcracker* by Tchaikovsky.

We can listen to the music as well as display the sound envelope.

In [ ]:
duration = 30  # in seconds
signal, sampling_rate = librosa.load(librosa.ex("nutcracker"), duration=duration)

# listen to the music
display(Audio(data=signal, rate=sampling_rate))

# look at the envelope
fig, ax = fig_ax()
ax.plot(np.arange(signal.size) / sampling_rate, signal)
ax.set_xlim(0, signal.size / sampling_rate)
ax.set_xlabel("Time (s)")
_ = ax.set(title="Sound envelope")

## Transform the signal

### Chromagram

In [ ]:
# Number of samples between successive chroma frames
hop_length = 128
# Number of chroma bins to produce
n_chroma = 12
# Compute the chromagram
chromagram = librosa.feature.chroma_cqt(
    y=signal, sr=sampling_rate, hop_length=hop_length, n_chroma=n_chroma
)

In [ ]:
fig, ax = fig_ax()
_ = librosa.display.specshow(
    chromagram,
    ax=ax,
    x_axis="s",
    y_axis="chroma",
    hop_length=hop_length,
    sr=sampling_rate,
)

### Tempogram

In [ ]:
# Compute the onset strength
hop_length = 256
oenv = librosa.onset.onset_strength(y=signal, sr=sampling_rate, hop_length=hop_length)
tempogram = librosa.feature.tempogram(
    onset_envelope=oenv,
    sr=sampling_rate,
    hop_length=hop_length,
)

In [ ]:
fig, ax = fig_ax()
_ = librosa.display.specshow(
    tempogram,
    ax=ax,
    hop_length=hop_length,
    sr=sampling_rate,
    x_axis="s",
    y_axis="tempo",
    cmap="magma",
)

## Signal segmentation

In [ ]:
chromagram.T.shape

In [ ]:
sigma = np.sqrt(chromagram.T.var(axis=0).mean())
n_samples, n_dims = chromagram.T.shape
pen_bic = 2 * sigma * sigma * np.log(n_samples) * (n_dims + 1)

In [ ]:
algo = rpt.KernelCPD(kernel="linear").fit(chromagram.T.astype(float))

In [ ]:
algo = rpt.KernelCPD(kernel="linear").fit(np.random.randn(1000, 12))

In [ ]:
chromagram.astype(np.double).dtype

In [ ]:
bkps = algo.predict(pen=pen_bic)
print(bkps)

In [ ]:
plt.plot(
    range(1, 21), [algo.cost.sum_of_costs(algo.predict(k)) for k in range(1, 21)], "*-"
)

In [ ]:
fig, ax = fig_ax()
_ = librosa.display.specshow(
    chromagram,
    ax=ax,
    x_axis="s",
    y_axis="chroma",
    hop_length=hop_length,
    sr=sampling_rate,
)

bkps_times = librosa.frames_to_time(bkps, sr=sampling_rate, hop_length=hop_length)
for b in bkps_times[:-1]:
    ax.axvline(b, ls="--", color="white", lw=4)

In [ ]:
def plot_util(
    rpt_bkps,
    feature,
    sampling_rate,
    hop_length,
    y_axis,
    vline_min,
    vline_max,
    ax,
    title,
):
    # Make the break points "compatible" with librosa api
    # it expects index 0 and not index of the last point
    rpt_bkps.pop(-1)
    rpt_bkps = [0] + rpt_bkps
    # Translate index in the chromagram to time index in the original soundtrack
    rpt_bpks_times = librosa.frames_to_time(rpt_bkps, sr=sampling_rate)
    # Plot
    librosa.display.specshow(
        chromagram, y_axis="chroma", x_axis="time", ax=axs[idx], hop_length=hop_length
    )
    axs[idx].vlines(
        rpt_bpks_times,
        0,
        chromagram.shape[0],
        color="linen",
        linestyle="--",
        linewidth=1.5,
        alpha=0.9,
    )
    axs[idx].set(title=f"Algo : {type(algo).__name__}, Cost : {cost}")
    axs[idx].label_outer()


algo = rpt.BottomUp(model="l2").fit(chromagram.T)
rpt_bkps = algo.predict(pen=50)
# Make the break points "compatible" with librosa api
# it expects index 0 and not index of the last point
rpt_bkps.pop(-1)
rpt_bkps = [0] + rpt_bkps
# Translate index in the chromagram to time index in the original soundtrack
rpt_bpks_times = librosa.frames_to_time(rpt_bkps, sr=sampling_rate)
# Plot
librosa.display.specshow(
    chromagram,
    y_axis="chroma",
    x_axis="time",
    ax=axs[n_plots - 1],
    hop_length=hop_length,
)
axs[n_plots - 1].vlines(
    rpt_bpks_times,
    0,
    chromagram.shape[0],
    color="linen",
    linestyle="--",
    linewidth=1.5,
    alpha=0.9,
)
axs[n_plots - 1].set(title=f"Algo : {type(algo).__name__}, Cost : l2")
axs[n_plots - 1].label_outer()

In [ ]:
# We ask for 5 breaking points
n_bkps = 5

algos = [rpt.BottomUp, rpt.Binseg]
models = ["l2", "l1", "rbf"]  # "l1", "rbf", "linear", "normal", "ar"
kernelCPD_algos = [rpt.KernelCPD]
kernelCPD_kernels = ["linear", "rbf", "cosine"]

model_objs = [(algo(model=model), model) for (algo, model) in product(algos, models)]
for (algo, kernel) in product(kernelCPD_algos, kernelCPD_kernels):
    model_objs.append((algo(kernel=kernel), kernel))

n_plots = len(models) * len(algos) + len(kernelCPD_kernels) * len(kernelCPD_algos) + 3
fig, axs = plt.subplots(nrows=n_plots, sharex=True, figsize=(8, 20), dpi=150)

times = librosa.times_like(oenv, sr=sampling_rate, hop_length=hop_length)
librosa.display.waveplot(y, sr=sampling_rate, ax=axs[0])
axs[0].set(title="nutcracker")
axs[0].label_outer()
axs[1].plot(times, oenv, label="Onset strength")
axs[1].label_outer()
axs[1].legend(frameon=True)

for idx, (algo, cost) in zip(range(2, n_plots), model_objs):
    algo = algo.fit(tempogram.T)
    rpt_bkps = algo.predict(n_bkps=n_bkps)
    # Make the break points "compatible" with librosa api
    # it expects index 0 and not index of the last point
    rpt_bkps.pop(-1)
    rpt_bkps = [0] + rpt_bkps
    # Translate index in the chromagram to time index in the original soundtrack
    rpt_bpks_times = librosa.frames_to_time(rpt_bkps, sr=sampling_rate)
    # Plot
    librosa.display.specshow(
        tempogram,
        y_axis="tempo",
        x_axis="time",
        ax=axs[idx],
        hop_length=hop_length,
        sr=sampling_rate,
        cmap="magma",
    )
    axs[idx].vlines(
        rpt_bpks_times, 16, 480, color="linen", linestyle="--", linewidth=1.5, alpha=0.9
    )
    axs[idx].set(title=f"Algo : {type(algo).__name__}, Cost : {cost}")
    axs[idx].label_outer()

algo = rpt.BottomUp(model="l2").fit(chromagram.T)
rpt_bkps = algo.predict(pen=50)
# Make the break points "compatible" with librosa api
# it expects index 0 and not index of the last point
# Translate index in the chromagram to time index in the original soundtrack
rpt_bpks_times = librosa.frames_to_time([0] + rpt_bkps[:-1], sr=sampling_rate)
# Plot
librosa.display.specshow(
    tempogram,
    y_axis="tempo",
    x_axis="time",
    ax=axs[n_plots - 1],
    hop_length=hop_length,
    sr=sampling_rate,
    cmap="magma",
)
axs[n_plots - 1].vlines(
    rpt_bpks_times, 16, 480, color="linen", linestyle="--", linewidth=1.5, alpha=0.9
)
axs[n_plots - 1].set(title=f"Algo : {type(algo).__name__}, Cost : l2")
axs[n_plots - 1].label_outer()

## Study each segment independently

Each segment of the music can be characterized by a set of features.

In [ ]:
sampling_rate, bkps_times / sampling_rate

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

for (segment_number, segment) in enumerate(
    np.split(chromagram.T, rpt_bkps[:-1]), start=1
):
    ax.plot(segment.mean(axis=0), "-*", label=f"Segment n°{segment_number}")
_ = plt.legend()


bkps_times = (rpt_bpks_times * sampling_rate).astype(int)[1:]
for (segment_number, segment) in enumerate(np.split(y, bkps_times), start=1):
    print(f"Segment n°{segment_number}")
    display(Audio(data=segment, rate=sampling_rate))